
## APLICACIÓN DE MACHINE LEARNING PARA CATEGORIZACIÓN DE CASOS DE LA MESA DE SERVICIO DE TECNOLOGÍA DE LA UNIVERSIDAD DE LOS ANDES

Machine Learning Techniques - Proyecto
Segundo Semestre - 2021

### Integrantes

*   Laura Natalia González García
*   Nicolás Gustavo Gaitán Gómez
*   David Camilo Bonilla Verdugo

### Problema
Dentro de los procesos de la DSIT, la categorización de casos se realiza de forma manual. Esto representa el trabajo de una persona a tiempo completo en días no pico de operación, y el apoyo de personal en días pico. Esta categorización, además, está sujeta al criterio de la persona que clasifica los casos entre solicitud o incidente, y la interpretación del mensaje para asociar el caso a un servicio, subservicio y categoría. Actualmente, el error de clasificación manual es de aproximadamente el 40%. Esta mala clasificación requiere una recategorización por parte de los ingenieros que atienden los casos, lo cual genera una demora en la respuesta de cada caso, debido al retraso en llegar al ingeniero adecuado. Dicho lo anterior, la DSIT requiere encontrar alternativas que permitan mejorar la eficiencia y efectividad del proceso de categorización de casos, de tal manera que la recategorización necesaria sea la menor posible, y el tiempo de espera para resolver un ticket sea el menor posible.

In [36]:
import nltk
#nltk.download() #Seleccionar "d" en Downloader y "all" en Identifiers

In [12]:
#Data management Libraries
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import glob
import six
sys.modules['sklearn.externals.six'] = six

# Data Modeling Libraries
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, MaxAbsScaler, LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix, mean_squared_error, r2_score, plot_precision_recall_curve
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.decomposition import TruncatedSVD
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier, MLPRegressor
import fasttext

#from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, recall_score

# Data visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Text preprocessing
import re, string, unicodedata
import inflect
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
pd.options.mode.chained_assignment = None
import spacy
import es_core_news_sm

#Save the model
from joblib import dump, load

## 1. Perfilamiento y Entendimiento de los datos

### Lectura de los datos

In [13]:
#Lectura de datos originales en Excel
data = pd.read_excel("/Users/laura/OneDrive - Universidad de los Andes/Cursos/MachineLearning/Proyecto/DSIT_DB_helpdesk.xlsx")

In [91]:
#Exportación de datos a CSV
data.to_csv("/Users/laura/OneDrive - Universidad de los Andes/Cursos/MachineLearning/MLTechniques/Proyecto/DSIT_DB_helpdesk_clean.csv")

### Entendimiento de los datos

In [7]:
data.head()

,Propietario del caso,Asunto,Fecha/Hora de apertura,Abierto,Cerrado,Cedex Responsable,Servicio,Sub Servicio,Categoria,Empleado Académico,...,Egresado,Afiliación Académica,Afiliación Administrativa,Afiliacion Egresado,Número del caso,Origen del caso,Estado,Descripción,Tipo,Antigüedad (Días)
0,Juan David Rodriguez Camacho,Ayuda con cuenta institucional,2021-01-25 09:56:00,0,1,CedEx Middlewere,Correo y Cuenta Uniandes,Inconvenientes Cuenta Uniandes,Problema al Ingresar,0,...,1,EDUCACION CONTINUADA,FACULTAD DE CIENCIAS SOCIALES,Maestría en Literatura,83168,Email,Cerrado,"Buenos días, Solicito su colaboración con la c...",Incidente,0
1,Edwin Orlando Herrera Lara,No deja ingreso,2021-01-27 19:30:00,0,1,CedEx Transformacion de procesos con TI,Soluciones para Automatización de procesos,Gestión de Errores técnicos en Procesos Automa...,Errores de Login,0,...,0,GOBIERNO Y ASUNTOS PUBLICOS,NaN,NaN,84969,Phone,Cerrado,-----Mensaje original----- De: Juan Esteban Ca...,Incidente,0
2,Juan Carlos Arévalo Jimenéz,En brightspace no puedo ver mis cursos,2021-01-22 23:56:00,0,1,NaN,NaN,NaN,NaN,0,...,0,ECONOMIA,NaN,NaN,82758,Mesa de Servicios Uniandes,Cerrado,Ademas de que tuve inconvenientes para ingresa...,Incidente,0
3,Edwin Orlando Herrera Lara,Banner,2021-01-13 15:16:00,0,1,CedEx Soluciones Académico-Administrativas,Servicios Académico Administrativo,Incidentes en Servicios Especializados,Problemas Banner,0,...,0,CIENCIA POLITICA,NaN,NaN,79778,Email,Cerrado,"Buenas tardes, No está funcionando banner para...",Incidente,0
4,Edwin Orlando Herrera Lara,Inconveniente con la cuenta de Zoom,2021-01-29 18:33:00,0,1,CedEx Comunicaciones y colaboración,Comunicación y Colaboración,"Reuniones virtuales (Teams, Webex, Zoom, telep...",Problema de acceso,0,...,0,NaN,NaN,NaN,85738,Email,Cerrado,"Buenas tardes, Actualmente, intenté usar mi cu...",Incidente,0


In [14]:
ProfileReport(data)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,Abierto,Cerrado,Empleado Académico,Empleado Administrativo,Estudiante,Egresado,Número del caso,Antigüedad (Días)
Abierto,1.000000,-1.000000,-0.008431,0.066826,-0.045499,-0.007814,0.192721,0.188385
Cerrado,-1.000000,1.000000,0.008431,-0.066826,0.045499,0.007814,-0.192721,-0.188385
Empleado Académico,-0.008431,0.008431,1.000000,-0.237153,-0.143768,0.174349,-0.035836,0.013707
Empleado Administrativo,0.066826,-0.066826,-0.237153,1.000000,-0.379277,0.096540,0.049798,0.094196
Estudiante,-0.045499,0.045499,-0.143768,-0.379277,1.000000,-0.085400,-0.068408,-0.039064
Egresado,-0.007814,0.007814,0.174349,0.096540,-0.085400,1.000000,-0.035586,0.011087
Número del caso,0.192721,-0.192721,-0.035836,0.049798,-0.068408,-0.035586,1.000000,-0.023556
Antigüedad (Días),0.188385,-0.188385,0.013707,0.094196,-0.039064,0.011087,-0.023556,1.000000


In [10]:
data['Tipo'].value_counts()

Solicitud    24785
Incidente    15496
Name: Tipo, dtype: int64

In [11]:
data['Servicio'].value_counts()

Correo y Cuenta Uniandes                                                            11018
Entornos de trabajo (tiempos sujetos a variación por protocolos de bioseguridad)     7755
Plataformas de enseñanza virtual                                                     5831
Servicios Administrativos                                                            2900
Comunicación y Colaboración                                                          2410
Servicios Académico Administrativo                                                   1332
Soluciones para Automatización de procesos                                           1195
Seguimiento y Control del Uso de Licencias de Software                               1077
Sitios Web y Apps                                                                     891
Servicios de Plataformas de Capa Media                                                830
Seguridad                                                                             680
Cómputo de

In [12]:
data['Sub Servicio'].value_counts()

Soporte Computador                                                       5686
Inconvenientes Cuenta Uniandes                                           3185
Inconvenientes con Correo electrónico/Listas de Correo/A2P/Calendario    2895
Correo Electrónico/Listas de Correo/A2P/Calendarios                      2610
Cuenta Uniandes                                                          2321
                                                                         ... 
Subservicio pruebas                                                         1
RADICACIÓN DE UNA QUEJA O RECLAMO                                           1
Apoyo Pago a Proveedores                                                    1
Atención de Incidentes Funcionales                                          1
Maestría en Ingeniería de Software                                          1
Name: Sub Servicio, Length: 118, dtype: int64

In [13]:
data['Categoria'].value_counts()

Instalar/ Actualizar/ Probar Software Institucional                       3443
Software                                                                  1695
Acceso, asignación, eliminación o modificación de secciones de usuario    1457
Modificaciones Cuenta                                                     1251
Correo Alterno                                                             989
                                                                          ... 
Incidente con reglas en Balanceador                                          1
Reportes de la BAM                                                           1
Generación de estado de cuenta                                               1
Validar funcionalidad en reportes de Facultades                              1
Información General                                                          1
Name: Categoria, Length: 348, dtype: int64

En algunos registros se observan datos faltanes o de prueba que deben ser eliminados del dataset. Los servicios, subservicios y categorías de bajo conteo (<10) deben ser agrupados como "Otros". Algunas de las columnas de datos como identificador del caso, afiliación del usuario, ingeniero responsable del caso, entre otros, no aportan información a la categorización por lo que deben ser eliminados.

## 2. Preprocesamiento de los datos

### Limpieza de datos
Para la construcción del modelo eliminamos los campos que no aportan información al caso, como la afiliación del usuario, propietario del caso, fecha y hora de apertura, número del caso, origen. Adicionalmente, filtramos por los casos que se encuentran en estado cerrado ya que son los que están correctamente categorizados y procesados por un agente de la mesa de servicios. Finalmente, eliminamos las filas con campos vacíos y separamos las variables de entrenamiento "x", de las que vamos a predecir "y" en varios datasets para darle diferente granularidad a los modelos.

In [4]:
#Hacemos una copia de los datos para evitar sobreescribirlos
df = data.copy()

In [5]:
#Filtramos los casos cerrados
df = df[df['Estado']=='Cerrado']
#Eliminamos las columnas que no aportan información para la clasificación del caso
df = df.drop(['Propietario del caso', 'Estado', 'Fecha/Hora de apertura', 'Cedex Responsable', 'Afiliación Académica','Afiliación Administrativa','Afiliacion Egresado', 'Número del caso', 'Origen del caso', 'Antigüedad (Días)','Abierto','Cerrado'], axis=1)
#Eliminamos las filas con campos vacíos
df = df.dropna()
df.head()

,Asunto,Servicio,Sub Servicio,Categoria,Empleado Académico,Empleado Administrativo,Estudiante,Egresado,Descripción,Tipo
0,Ayuda con cuenta institucional,Correo y Cuenta Uniandes,Inconvenientes Cuenta Uniandes,Problema al Ingresar,0,1,1,1,"Buenos días, Solicito su colaboración con la c...",Incidente
1,No deja ingreso,Soluciones para Automatización de procesos,Gestión de Errores técnicos en Procesos Automa...,Errores de Login,0,0,1,0,-----Mensaje original----- De: Juan Esteban Ca...,Incidente
3,Banner,Servicios Académico Administrativo,Incidentes en Servicios Especializados,Problemas Banner,0,0,1,0,"Buenas tardes, No está funcionando banner para...",Incidente
4,Inconveniente con la cuenta de Zoom,Comunicación y Colaboración,"Reuniones virtuales (Teams, Webex, Zoom, telep...",Problema de acceso,0,0,1,0,"Buenas tardes, Actualmente, intenté usar mi cu...",Incidente
5,Inscripción,Plataformas de enseñanza virtual,Problemas/Error de Acceso,Problemas al ingresar,0,0,1,0,"Cordial saludo, Estudiante reporta que no esta...",Incidente


In [6]:
#Eliminamos datos que no aportan información (e.g. Pruebas, Servicios de menos de 10 casos)
df = df[df['Servicio'] != 'Servicio pruebas']
df = df[df['Servicio'] != 'Gestión Presupuestal y Modelaje Financiero']
df = df[df['Servicio'] != 'Capacitaciones']
df = df[df['Servicio'] != 'Contratación Civil']
df = df[df['Servicio'] != 'Reportes e Información']
df = df[df['Servicio'] != 'Servicios de TV']
df = df[df['Servicio'] != 'Platypus']
df = df[df['Servicio'] != 'Sisinfo']
df = df[df['Servicio'] != 'Caracterización - Encuesta covid']

In [7]:
# Conteo de Tipo de Servicio después de la limpieza de datos
df['Tipo'].value_counts()

Solicitud    20769
Incidente    14226
Name: Tipo, dtype: int64

In [8]:
# Conteo de Servicios después de la limpieza de datos
df['Servicio'].value_counts()

Correo y Cuenta Uniandes                                                            10563
Entornos de trabajo (tiempos sujetos a variación por protocolos de bioseguridad)     6727
Plataformas de enseñanza virtual                                                     5139
Servicios Administrativos                                                            2738
Comunicación y Colaboración                                                          2342
Soluciones para Automatización de procesos                                           1178
Servicios Académico Administrativo                                                   1116
Seguimiento y Control del Uso de Licencias de Software                               1070
Sitios Web y Apps                                                                     818
Servicios de Plataformas de Capa Media                                                809
Seguridad                                                                             656
Cómputo de

In [172]:
#Carga de datos ya filtrados desde url
url = "https://raw.githubusercontent.com/dcbonilla10/MLTechniques/548a8ba0e2dc0b8d1afab50d94f25595eab96b6f/Proyecto/DSIT_DB_helpdesk_clean.csv"
df = pd.read_csv(url, index_col=0)
df2 = pd.read_csv(url, index_col=0)

In [94]:
#Separamos x y y para tres modelos con salidas diferentes.
df['AsDesc'] = df['Asunto']+' '+df['Descripción ']
X = df[['AsDesc']]
#y como Tipo
y_m1 = df['Tipo']
#y como Servicio
y_m2 = df['Servicio']

In [95]:
#Modelo 1: Entrenamiento de clasificador para predecir "Incidente o Solicitud"
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y_m1,stratify=y_m1,test_size=0.2, random_state=33)

In [96]:
#Modelo 2: Entrenamiento de clasificador para predecir "Servicio"
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y_m2,stratify=y_m2,test_size=0.2, random_state=33)

Adicionalmente haremos un alistamiento de los datos para usar el modelo pre-entrenado de fasttext (https://fasttext.cc/) como alternativa de transferencia de aprendizaje

In [173]:
#Organizamos los campos para FastText
df2['Servicio'] = df2['Servicio'].str.replace(' ','_')

In [174]:
df2['Servicio'] = "__label__" + df2['Servicio']
df2['Tipo'] = "__label__" + df2['Tipo']

In [372]:
#Separamos x y y para dos modelos con salidas diferentes.
df2['AsDesc'] = df2['Asunto']+' '+df2['Descripción ']

X2 = df2[['AsDesc']]
#y como Tipo
y_m1_Ft = df2['Tipo']
#y como Servicio
y_m2_Ft = df2['Servicio']

In [212]:
#Modelo 1: Entrenamiento de clasificador para predecir "Incidente o Solicitud"
X_train1_ft, X_test1_ft, y_train1_ft, y_test1_ft = train_test_split(X2,y_m1_Ft,stratify=y_m1_Ft,test_size=0.4, random_state=33)
X_val1_ft, X_test1_ft, y_val1_ft, y_test1_ft = train_test_split(X_test1_ft,y_test1_ft,stratify=y_test1_ft,test_size=0.5, random_state=33)

In [357]:
#Modelo 2: Entrenamiento de clasificador para predecir "Servicio
X_train2_ft, X_test2_ft, y_train2_ft, y_test2_ft = train_test_split(X2,y_m2_Ft,stratify=y_m2_Ft,test_size=0.4, random_state=33)
X_val2_ft, X_test2_ft, y_val2_ft, y_test2_ft = train_test_split(X_test2_ft,y_test2_ft,stratify=y_test2_ft,test_size=0.5, random_state=33)


In [299]:
tot1_ft = df2['Tipo'] + ' ' + df2['AsDesc']
tot2_ft = df2['Servicio'] + ' ' + df2['AsDesc']
train1_ft = y_train1_ft + ' ' + X_train1_ft['AsDesc']
train2_ft = y_train2_ft + ' ' + X_train2_ft['AsDesc']
val1_ft = y_val1_ft + ' ' + X_val1_ft['AsDesc']
val2_ft = y_val2_ft + ' ' + X_val2_ft['AsDesc']
test1_ft = y_test1_ft + ' ' + X_test1_ft['AsDesc']
test2_ft = y_test2_ft + ' ' + X_test2_ft['AsDesc']

In [304]:
train1_ft.head()

19101    __label__Solicitud instalación microsoft visio...
28221    __label__Solicitud activación autenticación  p...
37950    __label__Solicitud creación cuenta impresional...
2176     __label__Solicitud validar mensaje de actualiz...
168      __label__Incidente error acceso cuenta buenos ...
dtype: object

In [305]:
train2_ft.head()

35626    __label__Plataformas_de_enseñanza_virtual dese...
17174    __label__Entornos_de_trabajo_(tiempos_sujetos_...
17571    __label__Sitios_Web_y_Apps activación de githu...
25923    __label__Correo_y_Cuenta_Uniandes correo alter...
19676    __label__Servicios_Administrativos recuperar c...
dtype: object

In [370]:
#Guardado de datasets para entrenar modelo fasttext
import csv as csv2
train1_ft.to_csv('train1_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')
train2_ft.to_csv('train2_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')
test1_ft.to_csv('test1_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')
test2_ft.to_csv('test2_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')
val1_ft.to_csv('val1_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')
val2_ft.to_csv('val2_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')
tot1_ft.to_csv('tot1_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')
tot2_ft.to_csv('tot2_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')


### Preprocesamiento

Para poder realizar el pre-procesamiento de los datos, es recomendable pasar por tres etapas:
* Eliminación del Ruido.
* Tokenización.
* Normalización.
Para entrenar los modelos de SVM, NB y Reg Logística, correremos los tres pasos. Para el modelo de redes neuronales de fasttext solamente haremos la eliminación del ruido, ya que este algoritmo incorpora un proceso interno de tokenizacion y normalización basado en skipgrams.

##### ** Eliminación del Ruido**
La eliminación del ruido se utiliza para dejar el archivo en texto plano, sobre todo cuando vienen de diferentes fuentes como HTML, Hashtags, XML, entre otros. También para eliminar caracteres especiales y pasar todo a minúscula.

In [97]:
X_train1.head()

,AsDesc
30369,Matricula No me deja actualizar el perfil para...
37520,Activar notas parciales Buen día. Favor proces...
15673,Licencia STATA Buenas Tardes La usuaria Olga L...
30144,Cambio nombre cuenta corporativa CPOL Estimado...
33529,Solicitudes Hola buen día cordial saludo la se...


In [98]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
        elif word == 'no':
            new_words.append(word)
    return new_words

def preprocessing(df):
    #Reemplazar URLs por url
    df['preprocessed_ad'] = df['AsDesc'].str.replace('http\S+|www.\S+', 'url', case=False)
    #Eliminar referencia a imagenes
    df['preprocessed_ad'] = df['preprocessed_ad'].str.replace('\[cid\S+', '', case=False)
    #Reemplazar emails por mail
    df['preprocessed_ad'] = df['preprocessed_ad'].str.replace('\S+@\S+', 'mail', case=False)
    #Pasar a minúsculas
    df['preprocessed_ad'] = df['preprocessed_ad'].str.lower()
    #Eliminar puntuación
    df['preprocessed_ad'] = df['preprocessed_ad'].str.replace('[^\w\s]', '', case=False)
    #Eliminar números
    df['preprocessed_ad'] = df['preprocessed_ad'].str.replace('\d+', '', case=False) 
    return df

In [368]:
X_train1_ft = preprocessing(X_train1_ft)
X_train2_ft = preprocessing(X_train2_ft)
X_test1_ft = preprocessing(X_test1_ft)
X_test2_ft = preprocessing(X_test2_ft)
X_val1_ft = preprocessing(X_val1_ft)
X_val2_ft = preprocessing(X_val2_ft)

In [369]:
#Para los datos de fasttext, unimos nuevamente los datos preprocesados con las etiquetas
train1_ft = y_train1_ft + ' ' + X_train1_ft['preprocessed_ad']
train2_ft = y_train2_ft + ' ' + X_train2_ft['preprocessed_ad']
val1_ft = y_val1_ft + ' ' + X_val1_ft['preprocessed_ad']
val2_ft = y_val2_ft + ' ' + X_val2_ft['preprocessed_ad']
test1_ft = y_test1_ft + ' ' + X_test1_ft['preprocessed_ad']
test2_ft = y_test2_ft + ' ' + X_test2_ft['preprocessed_ad']

In [247]:
X_train1 = preprocessing(X_train1)
X_train2 = preprocessing(X_train2)
X_train1.head()

,AsDesc,preprocessed_ad
30369,Matricula No me deja actualizar el perfil para...,matricula no me deja actualizar el perfil para...
37520,Activar notas parciales Buen día. Favor proces...,activar notas parciales buen día favor procesa...
15673,Licencia STATA Buenas Tardes La usuaria Olga L...,licencia stata buenas tardes la usuaria olga l...
30144,Cambio nombre cuenta corporativa CPOL Estimado...,cambio nombre cuenta corporativa cpol estimado...
33529,Solicitudes Hola buen día cordial saludo la se...,solicitudes hola buen día cordial saludo la se...


In [99]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
        elif word == 'no':
            new_words.append(word)
    return new_words

def preprocessing2(words):
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

##### ** Tokenización**
La tokenización permite dividir frases u oraciones en palabras. Con el fin de desglozar las palabras correctamente para el posterior análisis. Pero primero, se realiza una corrección de las contracciones que pueden estar presentes en los textos.

In [251]:
#Tokenización de asunto y descripción
X_train1['tokenized_ad'] = X_train1.apply(lambda row: nltk.word_tokenize(row['preprocessed_ad']), axis=1)
X_train2['tokenized_ad'] = X_train2.apply(lambda row: nltk.word_tokenize(row['preprocessed_ad']), axis=1)
X_train1.head()

,AsDesc,preprocessed_ad,tokenized_ad
30369,Matricula No me deja actualizar el perfil para...,matricula no me deja actualizar el perfil para...,"[matricula, no, me, deja, actualizar, el, perf..."
37520,Activar notas parciales Buen día. Favor proces...,activar notas parciales buen día favor procesa...,"[activar, notas, parciales, buen, día, favor, ..."
15673,Licencia STATA Buenas Tardes La usuaria Olga L...,licencia stata buenas tardes la usuaria olga l...,"[licencia, stata, buenas, tardes, la, usuaria,..."
30144,Cambio nombre cuenta corporativa CPOL Estimado...,cambio nombre cuenta corporativa cpol estimado...,"[cambio, nombre, cuenta, corporativa, cpol, es..."
33529,Solicitudes Hola buen día cordial saludo la se...,solicitudes hola buen día cordial saludo la se...,"[solicitudes, hola, buen, día, cordial, saludo..."


In [254]:
#Remoción de stopwords y non-ascii
X_train1['preprocessed_ad'] = X_train1.apply(lambda row: " ".join(preprocessing2(row['tokenized_ad'])),axis=1)
X_train2['preprocessed_ad'] = X_train2.apply(lambda row: " ".join(preprocessing2(row['tokenized_ad'])),axis=1)
X_train1.head()

,AsDesc,preprocessed_ad,tokenized_ad
30369,Matricula No me deja actualizar el perfil para...,matricula no deja actualizar perfil matricula ...,"[matricula, no, me, deja, actualizar, el, perf..."
37520,Activar notas parciales Buen día. Favor proces...,activar notas parciales buen dia favor procesa...,"[activar, notas, parciales, buen, día, favor, ..."
15673,Licencia STATA Buenas Tardes La usuaria Olga L...,licencia stata buenas tardes usuaria olga luci...,"[licencia, stata, buenas, tardes, la, usuaria,..."
30144,Cambio nombre cuenta corporativa CPOL Estimado...,cambio nombre cuenta corporativa cpol estimado...,"[cambio, nombre, cuenta, corporativa, cpol, es..."
33529,Solicitudes Hola buen día cordial saludo la se...,solicitudes hola buen dia cordial saludo seman...,"[solicitudes, hola, buen, día, cordial, saludo..."


#####  ** Normalización**
Para la normalización de los datos se realiza una eliminación de prefijos y sufijos, además de realizar una lemmatización de los verbos.

In [262]:
 #Aplica lematización
nlp = spacy.load("es_core_news_sm")
X_train1['lem_ad'] = X_train1['preprocessed_ad'].apply(lambda row: " ".join([w.lemma_ for w in nlp(str(row))]))
X_train2['lem_ad'] = X_train2['preprocessed_ad'].apply(lambda row: " ".join([w.lemma_ for w in nlp(str(row))]))

X_train1.head()

,AsDesc,preprocessed_ad,tokenized_ad,lem_ad
30369,Matricula No me deja actualizar el perfil para...,matricula no deja actualizar perfil matricula ...,"[matricula, no, me, deja, actualizar, el, perf...",matricula no dejar actualizar perfil matricula...
37520,Activar notas parciales Buen día. Favor proces...,activar notas parciales buen dia favor procesa...,"[activar, notas, parciales, buen, día, favor, ...",activar nota parcial buen dia favor procesar s...
15673,Licencia STATA Buenas Tardes La usuaria Olga L...,licencia stata buenas tardes usuaria olga luci...,"[licencia, stata, buenas, tardes, la, usuaria,...",licencia statar buena tarde usuario olgo lucia...
30144,Cambio nombre cuenta corporativa CPOL Estimado...,cambio nombre cuenta corporativa cpol estimado...,"[cambio, nombre, cuenta, corporativa, cpol, es...",cambio nombre contar corporativo cpol estimado...
33529,Solicitudes Hola buen día cordial saludo la se...,solicitudes hola buen dia cordial saludo seman...,"[solicitudes, hola, buen, día, cordial, saludo...",solicitud hola buen dia cordial saludo semana ...


In [101]:
nlp = spacy.load("es_core_news_sm")


In [103]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer('spanish')
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

In [266]:
#Stemming
X_train1['tokenized_ad'] = X_train1.apply(lambda row: nltk.word_tokenize(row['lem_ad']), axis=1)
X_train2['tokenized_ad'] = X_train2.apply(lambda row: nltk.word_tokenize(row['lem_ad']), axis=1)
X_train1['stem_ad'] = X_train1.apply(lambda row: stem_words(row['tokenized_ad']),axis=1)
X_train2['stem_ad'] = X_train2.apply(lambda row: stem_words(row['tokenized_ad']),axis=1)
X_train1.head()

,AsDesc,preprocessed_ad,tokenized_ad,lem_ad,stem_ad
30369,Matricula No me deja actualizar el perfil para...,matricula no deja actualizar perfil matricula ...,"[matricula, no, dejar, actualizar, perfil, mat...",matricula no dejar actualizar perfil matricula...,"[matricul, no, dej, actualiz, perfil, matricul..."
37520,Activar notas parciales Buen día. Favor proces...,activar notas parciales buen dia favor procesa...,"[activar, nota, parcial, buen, dia, favor, pro...",activar nota parcial buen dia favor procesar s...,"[activ, not, parcial, buen, dia, favor, proces..."
15673,Licencia STATA Buenas Tardes La usuaria Olga L...,licencia stata buenas tardes usuaria olga luci...,"[licencia, statar, buena, tarde, usuario, olgo...",licencia statar buena tarde usuario olgo lucia...,"[licenci, stat, buen, tard, usuari, olgo, luci..."
30144,Cambio nombre cuenta corporativa CPOL Estimado...,cambio nombre cuenta corporativa cpol estimado...,"[cambio, nombre, contar, corporativo, cpol, es...",cambio nombre contar corporativo cpol estimado...,"[cambi, nombr, cont, corpor, cpol, estim, equi..."
33529,Solicitudes Hola buen día cordial saludo la se...,solicitudes hola buen dia cordial saludo seman...,"[solicitud, hola, buen, dia, cordial, saludo, ...",solicitud hola buen dia cordial saludo semana ...,"[solicitud, hol, buen, dia, cordial, salud, se..."


In [267]:
#Guardar datos preprocesados
X_train1.to_csv("/Users/laura/OneDrive - Universidad de los Andes/Cursos/MachineLearning/MLTechniques/Proyecto/X_train1_stemmed.csv")
X_train2.to_csv("/Users/laura/OneDrive - Universidad de los Andes/Cursos/MachineLearning/MLTechniques/Proyecto/X_train2_stemmed.csv")

In [74]:
#Guardar datos con stemming y lematización
X_train1 = pd.read_csv("/Users/laura/OneDrive - Universidad de los Andes/Cursos/MachineLearning/MLTechniques/Proyecto/X_train1_stemmed.csv", index_col=0)
X_train2 = pd.read_csv("/Users/laura/OneDrive - Universidad de los Andes/Cursos/MachineLearning/MLTechniques/Proyecto/X_train2_stemmed.csv", index_col=0)

In [75]:
X_train1_m.head() 

,stem_ad
30369,"['matricul', 'no', 'dej', 'actualiz', 'perfil'..."
37520,"['activ', 'not', 'parcial', 'buen', 'dia', 'fa..."
15673,"['licenci', 'stat', 'buen', 'tard', 'usuari', ..."
30144,"['cambi', 'nombr', 'cont', 'corpor', 'cpol', '..."
33529,"['solicitud', 'hol', 'buen', 'dia', 'cordial',..."


In [76]:
X_train1_m = X_train1[['stem_ad']]
X_train2_m = X_train2[['stem_ad']]

##### ** Transformación de campos**
Realizaremos la transformación de campos tf-idf (term-frequency times inverse document-frequency).

In [77]:
X_train1_text = X_train1_m['stem_ad']
X_train1_text = X_train1_text.apply(lambda x: ''.join(x))
X_train2_text = X_train2_m['stem_ad']
X_train2_text = X_train2_text.apply(lambda x: ''.join(x))
X_train1_text

30369    ['matricul', 'no', 'dej', 'actualiz', 'perfil'...
37520    ['activ', 'not', 'parcial', 'buen', 'dia', 'fa...
15673    ['licenci', 'stat', 'buen', 'tard', 'usuari', ...
30144    ['cambi', 'nombr', 'cont', 'corpor', 'cpol', '...
33529    ['solicitud', 'hol', 'buen', 'dia', 'cordial',...
                               ...                        
20358    ['creacion', 'cuent', 'educacion', 'ejecut', '...
26368    ['corre', 'uniand', 'contrasen', 'corre', 'alt...
32020    ['sal', 'error', 'intent', 'ingres', 'panopt',...
3839     ['solicitud', 'trasl', 'conten', 'curs', 'sicu...
3806     ['gestion', 'ingres', 'brightspac', 'buen', 't...
Name: stem_ad, Length: 27996, dtype: object

In [78]:
# Transformación tf-idf
tf_idf1 = TfidfVectorizer(analyzer='word', ngram_range=(1,1))
tf_idf2 = TfidfVectorizer(analyzer='word', ngram_range=(1,1))

X_train1_tf = tf_idf1.fit_transform(X_train1_text)
X_train2_tf = tf_idf2.fit_transform(X_train2_text)

print(X_train1_tf.shape)
print(X_train2_tf.shape)

(27996, 31345)
(27996, 31050)


##### ** Balance de clases**
Realizaremos el balanceo de clases usando SMOTE y RandomUnderSample. Se pueden balancear los datos transformados binarios, conteos o tf-idf. SMOTE funciona por supermuestreo (oversampling), igualando los conteos de las clases con menos datos, a la cantidad de datos de la clase mayoritaria. RandonUnderSample hace un submuestreo de la clase mayoritaria para igualar los datos a la clase minoritaria. 
Para este caso, haremos supermuestreo de las clases menores para que tengan al menos el valor promedio de datos de las clases, y haremos un submuestreo de las clases mayores para que tengan máximo el valor promedio de datos de las clases.

In [79]:
# Balance de clases
counter1 = Counter(y_train1)
counter2 = Counter(y_train2)

print("Balance de clases original")
print(counter1)
print(counter2)

average1 = 0
for val in counter1.values():
    average1 += val
average1 = np.floor(average1/2)
average1 = average1.astype(int)

average2 = 0
for val in counter2.values():
    average2 += val
average2 = np.floor(average2/15)
average2 = average2.astype(int)

print("Promedio de datos por clase")
print(average1)
print(average2)

Balance de clases original
Counter({'Solicitud': 16615, 'Incidente': 11381})
Counter({'Correo y Cuenta Uniandes': 8451, 'Entornos de trabajo (tiempos sujetos a variación por protocolos de bioseguridad)': 5382, 'Plataformas de enseñanza virtual': 4111, 'Servicios Administrativos': 2190, 'Comunicación y Colaboración': 1874, 'Soluciones para Automatización de procesos': 942, 'Servicios Académico Administrativo': 893, 'Seguimiento y Control del Uso de Licencias de Software': 856, 'Sitios Web y Apps': 654, 'Servicios de Plataformas de Capa Media': 647, 'Seguridad': 525, 'Cómputo de alto desempeño para Investigación': 524, 'Acceso a Internet y Redes Uniandes': 471, 'Soluciones académicas': 358, 'Servicios Nube': 118})
Promedio de datos por clase
13998
1866


In [80]:
#Pipeline de balanceo en pre-procesamiento
over = SMOTE(sampling_strategy = {'Incidente': average1})
under = RandomUnderSampler(sampling_strategy = {'Solicitud': average1})
steps = [('o', over), ('u', under)]
pipeBalance = Pipeline(steps=steps)

# Transformación (Se hace sobre x y y para conservar la relación entre los datos)
X_train1_tf_bal, y_train1_bal = pipeBalance.fit_resample(X_train1_tf, y_train1)

counter = Counter(y_train1_bal)
print("Balance de clases para tf-idf")
print(counter)

Balance de clases para tf-idf
Counter({'Incidente': 13998, 'Solicitud': 13998})


In [81]:
#Pipeline de balanceo en pre-procesamiento
over = SMOTE(sampling_strategy = {'Servicios Nube': average2,
                                  'Soluciones académicas': average2,
                                  'Acceso a Internet y Redes Uniandes': average2,
                                  'Cómputo de alto desempeño para Investigación':average2,
                                 'Seguridad':average2,
                                 'Servicios de Plataformas de Capa Media':average2,
                                 'Sitios Web y Apps':average2,
                                 'Seguimiento y Control del Uso de Licencias de Software':average2,
                                 'Soluciones para Automatización de procesos':average2,
                                 'Servicios Académico Administrativo': average2})
under = RandomUnderSampler(sampling_strategy = {'Correo y Cuenta Uniandes': average2, 
                                                'Entornos de trabajo (tiempos sujetos a variación por protocolos de bioseguridad)': average2, 
                                                'Plataformas de enseñanza virtual': average2,
                                                'Servicios Administrativos': average2, 
                                                'Comunicación y Colaboración': average2})
steps = [('o', over), ('u', under)]
pipeBalance = Pipeline(steps=steps)

# Transformación (Se hace sobre x y y para conservar la relación entre los datos)
X_train2_tf_bal, y_train2_bal = pipeBalance.fit_resample(X_train2_tf, y_train2)

counter = Counter(y_train2_bal)
print("Balance de clases para tf-idf")
print(counter)

Balance de clases para tf-idf
Counter({'Acceso a Internet y Redes Uniandes': 1866, 'Comunicación y Colaboración': 1866, 'Correo y Cuenta Uniandes': 1866, 'Cómputo de alto desempeño para Investigación': 1866, 'Entornos de trabajo (tiempos sujetos a variación por protocolos de bioseguridad)': 1866, 'Plataformas de enseñanza virtual': 1866, 'Seguimiento y Control del Uso de Licencias de Software': 1866, 'Seguridad': 1866, 'Servicios Académico Administrativo': 1866, 'Servicios Administrativos': 1866, 'Servicios Nube': 1866, 'Servicios de Plataformas de Capa Media': 1866, 'Sitios Web y Apps': 1866, 'Soluciones académicas': 1866, 'Soluciones para Automatización de procesos': 1866})


##### **Selección de variables**
Utilizamos la librería TrucatedSVD para reducir la dimensionalidad de los datos. Previamente se deben escalar los datos. Se fija el número de componentes en 2500 ya que con esta cantidad se explica un buen porcentaje de la varianza para la mayoría de los casos, sin embargo, se puede ajustar este valor e iterar hasta el total de componentes iniciales, si se tienen suficientes recursos computacionales. Para 2500 componentes se requieren 8Gb de RAM disponible aproximadamente. 

No se utiliza PCA porque no está diseñado para matrices dispersas y con matrices densas del tamaño del dataset requiere demasiada RAM (16GB para dos componentes), con lo cual no se logra una buena reducción de componentes que explique la varianza de manera adecuada.

In [38]:
scaler = MaxAbsScaler()
tsvd = TruncatedSVD(n_components=2500, random_state=42)

steps = [('s', scaler), ('t', tsvd)]
pipeDimension = Pipeline(steps=steps)

X_train1_tf_tsvd = pipeDimension.fit_transform(X_train1_tf)
print(f"Total varianza explicada para tf-idf: {np.sum(tsvd.explained_variance_ratio_):.2f}")
X_train1_tf_bal_tsvd = pipeDimension.fit_transform(X_train1_tf_bal)
print(f"Total varianza explicada para tf-idf balanceado: {np.sum(tsvd.explained_variance_ratio_):.2f}")

Total varianza explicada para tf-idf: 0.78
Total varianza explicada para tf-idf balanceado: 0.79


In [40]:
scaler = MaxAbsScaler()
tsvd = TruncatedSVD(n_components=2500, random_state=42)

steps = [('s', scaler), ('t', tsvd)]
pipeDimension = Pipeline(steps=steps)

X_train2_tf_tsvd = pipeDimension.fit_transform(X_train2_tf)
print(f"Total varianza explicada para tf-idf: {np.sum(tsvd.explained_variance_ratio_):.2f}")
X_train2_tf_bal_tsvd = pipeDimension.fit_transform(X_train2_tf_bal)
print(f"Total varianza explicada para tf-idf balanceado: {np.sum(tsvd.explained_variance_ratio_):.2f}")

Total varianza explicada para tf-idf: 0.78
Total varianza explicada para tf-idf balanceado: 0.85


In [106]:
#Estandarización de datos para eliminar valores negativos o fuera del rango 0-1 después de las transformaciones
scaler = MaxAbsScaler()
X_train1_tf = scaler.fit_transform(X_train1_tf)
X_train1_tf_bal = scaler.fit_transform(X_train1_tf_bal)
X_train1_tf_tsvd = scaler.fit_transform(X_train1_tf_tsvd)
X_train1_tf_bal_tsvd = scaler.fit_transform(X_train1_tf_bal_tsvd)

In [146]:
scaler = MaxAbsScaler()
X_train2_tf = scaler.fit_transform(X_train2_tf)
X_train2_tf_bal = scaler.fit_transform(X_train2_tf_bal)
X_train2_tf_tsvd = scaler.fit_transform(X_train2_tf_tsvd)
X_train2_tf_bal_tsvd = scaler.fit_transform(X_train2_tf_bal_tsvd)

### 4A. Modelamiento y Predicción - Tipo de Caso

### **4.1 Modelo Base**
Corremos modelos base utilizando los dataset creados

- Regresión logística

In [5]:
#RegLog
clf = LogisticRegression(max_iter=1000, n_jobs=6)
clf.fit(X_train1_tf, y_train1)
Y_train1_pred = clf.predict(X_train1_tf)

In [6]:
best_model=clf

In [7]:
report_LR = classification_report(y_train1, Y_train1_pred)
print("Test Report para RegLog \n", report_LR)
print("Confusion Matrix para RegLog\n", confusion_matrix(y_train1, Y_train1_pred))

Test Report para RegLog 
               precision    recall  f1-score   support

   Incidente       0.86      0.85      0.85     11381
   Solicitud       0.90      0.91      0.90     16615

    accuracy                           0.88     27996
   macro avg       0.88      0.88      0.88     27996
weighted avg       0.88      0.88      0.88     27996

Confusion Matrix para RegLog
 [[ 9619  1762]
 [ 1564 15051]]


- SVM: Sin balanceo de clases 

In [ ]:
#SVM no bal
svm = SVC()
svm.fit(X_train1_tf, y_train1)
Y_train1_pred_svm = svm.predict(X_train1_tf)

In [84]:
report_SVM = classification_report(y_train1, Y_train1_pred_svm)
print("Test Report para SVM \n", report_SVM)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train1, Y_train1_pred_svm))

Test Report para SVM 
               precision    recall  f1-score   support

   Incidente       0.91      0.91      0.91     11381
   Solicitud       0.94      0.94      0.94     16615

    accuracy                           0.92     27996
   macro avg       0.92      0.92      0.92     27996
weighted avg       0.92      0.92      0.92     27996

Confusion Matrix para SVM
 [[10329  1052]
 [ 1056 15559]]


- SVM: Con balanceo de clases 

In [8]:
#SVM bal
svm = SVC()
svm.fit(X_train1_tf_bal, y_train1_bal)
Y_train1_pred_svm_bal = svm.predict(X_train1_tf_bal)

In [9]:
report_SVM_bal = classification_report(y_train1_bal, Y_train1_pred_svm_bal)
print("Test Report para SVM \n", report_SVM_bal)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train1_bal, Y_train1_pred_svm_bal))

Test Report para SVM 
               precision    recall  f1-score   support

   Incidente       0.91      0.96      0.94     13998
   Solicitud       0.96      0.91      0.93     13998

    accuracy                           0.93     27996
   macro avg       0.94      0.93      0.93     27996
weighted avg       0.94      0.93      0.93     27996

Confusion Matrix para SVM
 [[13421   577]
 [ 1263 12735]]


- SVM: Con reduccion

In [10]:
#SVM reducido
svm = SVC()
svm.fit(X_train1_tf_tsvd, y_train1)
Y_train1_pred_svm_tsvd = svm.predict(X_train1_tf_tsvd)

In [11]:
report_SVM_tsvd = classification_report(y_train1, Y_train1_pred_svm_tsvd)
print("Test Report para SVM \n", report_SVM_tsvd)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train1, Y_train1_pred_svm_tsvd))

Test Report para SVM 
               precision    recall  f1-score   support

   Incidente       0.90      0.89      0.90     11381
   Solicitud       0.93      0.93      0.93     16615

    accuracy                           0.92     27996
   macro avg       0.92      0.91      0.92     27996
weighted avg       0.92      0.92      0.92     27996

Confusion Matrix para SVM
 [[10179  1202]
 [ 1087 15528]]


- SVM: Con balanceo de clases con reducción

In [29]:
#SVM bal reducido
svm = SVC()
svm.fit(X_train1_tf_bal, y_train1_bal)
Y_train1_pred_svm_tf_bal = svm.predict(X_train1_tf_bal)

In [30]:
report_SVM_tf_bal= classification_report(y_train1_bal, Y_train1_pred_svm_tf_bal)
print("Test Report para SVM \n", report_SVM_tf_bal)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train1_bal, Y_train1_pred_svm_tf_bal))

Test Report para SVM 
               precision    recall  f1-score   support

   Incidente       0.91      0.96      0.94     13998
   Solicitud       0.96      0.91      0.93     13998

    accuracy                           0.93     27996
   macro avg       0.94      0.93      0.93     27996
weighted avg       0.94      0.93      0.93     27996

Confusion Matrix para SVM
 [[13421   577]
 [ 1263 12735]]


- Clasificador bayesiano: sin balanceo de clases ni reduccion de dimensionalidad

In [110]:
#Bayes
nb = MultinomialNB()
nb.fit(X_train1_tf, y_train1)
Y_train1_pred_nb = nb.predict(X_train1_tf)

In [111]:
report_nb = classification_report(y_train1, Y_train1_pred_nb)
print("Test Report para NB \n", report_nb)
print("Confusion Matrix para NB\n", confusion_matrix(y_train1, Y_train1_pred_nb))

Test Report para NB 
               precision    recall  f1-score   support

   Incidente       0.77      0.78      0.78     11381
   Solicitud       0.85      0.84      0.85     16615

    accuracy                           0.82     27996
   macro avg       0.81      0.81      0.81     27996
weighted avg       0.82      0.82      0.82     27996

Confusion Matrix para NB
 [[ 8919  2462]
 [ 2627 13988]]


- Clasificador bayesiano: con balanceo de clases sin reduccion de dimensionalidad

In [112]:
#Bayes bal
nb = MultinomialNB()
nb.fit(X_train1_tf_bal, y_train1_bal)
Y_train1_pred_nb_bal = nb.predict(X_train1_tf_bal)

In [113]:
report_nb_bal = classification_report(y_train1_bal, Y_train1_pred_nb_bal)
print("Test Report para NB \n", report_nb_bal)
print("Confusion Matrix para NB\n", confusion_matrix(y_train1_bal, Y_train1_pred_nb_bal))

Test Report para NB 
               precision    recall  f1-score   support

   Incidente       0.80      0.86      0.83     13998
   Solicitud       0.85      0.79      0.82     13998

    accuracy                           0.82     27996
   macro avg       0.82      0.82      0.82     27996
weighted avg       0.82      0.82      0.82     27996

Confusion Matrix para NB
 [[12014  1984]
 [ 2997 11001]]


### 4.2 Optimización de hiperparámetros

Se realiza un procedimiento de optimización de hiperparámetros para regresión logística

In [23]:
pipe = Pipeline([('clf' , RandomForestClassifier())])

param_grid = [
    {'clf' : [LogisticRegression()],
     'clf__penalty' : ['l1', 'l2'],
    'clf__C' : np.logspace(-4, 4, 20),
    'clf__solver' : ['liblinear']},
    {'clf' : [RandomForestClassifier()],
    'clf__n_estimators' : list(range(10,101,10)),
    'clf__max_features' : list(range(6,32,5))}
]

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=6)
clf.fit(X_train1_tf, y_train1)
best_logistic_model = clf.best_estimator_
Y_train1_best_logistic_model_pred = best_logistic_model.predict(X_train1_tf)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [121]:
clf.best_params_

{'clf': LogisticRegression(C=0.615848211066026, penalty='l1', solver='liblinear'),
 'clf__C': 0.615848211066026,
 'clf__penalty': 'l1',
 'clf__solver': 'liblinear'}

In [24]:
report_LR_cv = classification_report(y_train1, Y_train1_best_logistic_model_pred)
print("Test Report para RegLog \n", report_LR_cv)
print("Confusion Matrix para RegLog\n", confusion_matrix(y_train1, Y_train1_best_logistic_model_pred))

Test Report para RegLog 
               precision    recall  f1-score   support

   Incidente       0.78      0.74      0.76     11381
   Solicitud       0.83      0.86      0.84     16615

    accuracy                           0.81     27996
   macro avg       0.80      0.80      0.80     27996
weighted avg       0.81      0.81      0.81     27996

Confusion Matrix para RegLog
 [[ 8369  3012]
 [ 2324 14291]]


Se realiza un procedimiento de optimización de hiperparámetros para SVM

In [87]:
pipe_SVM = Pipeline([('svm' , SVC())])
param_grid_SVM = dict(svm__C = [0.1, 10, 100],
               svm__gamma = ['scale'],
               svm__kernel = ['rbf', 'sigmoid'],
               )

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search_SVM = GridSearchCV(pipe_SVM, param_grid_SVM, cv = cv, verbose=True, n_jobs = -1,scoring='accuracy')

In [88]:
grid_search_SVM.fit(X_train1_tf_bal, y_train1_bal)
best_model_SVM = grid_search_SVM.best_estimator_
Y_pred_SVM_best_model = best_model_SVM.predict(X_train1_tf_bal)

Fitting 15 folds for each of 6 candidates, totalling 90 fits


In [89]:
grid_search_SVM.best_params_

{'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}

In [90]:
report_SVM_bal_best_model = classification_report(y_train1_bal, Y_pred_SVM_best_model)
print("Test Report para SVM \n", report_SVM_bal_best_model)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train1_bal, Y_pred_SVM_best_model))

Test Report para SVM 
               precision    recall  f1-score   support

   Incidente       1.00      1.00      1.00     13998
   Solicitud       1.00      1.00      1.00     13998

    accuracy                           1.00     27996
   macro avg       1.00      1.00      1.00     27996
weighted avg       1.00      1.00      1.00     27996

Confusion Matrix para SVM
 [[13967    31]
 [   63 13935]]


### 4.3 Entrenamiento Red Neuronal FastText

In [202]:
model_ft = fasttext.train_supervised(input='train1_ft.txt', autotuneValidationFile='val1_ft.txt')

In [203]:
model_ft.test("val1_ft.txt")

(3499, 0.7813661046013146, 0.7813661046013146)

### 4B. Modelamiento y Predicción - Servicio

### **4.1 Modelo Base**
Corremos modelos base utilizando los dataset creados

- Regresión logística

In [122]:
#RegLog
y_train2 = y_train2.str.replace('Entornos de trabajo \(tiempos sujetos a variación por protocolos de bioseguridad\)','Entornos de Trabajo')
clf = LogisticRegression(max_iter=1000, n_jobs=6)
clf.fit(X_train2_tf, y_train2)
Y_train2_pred = clf.predict(X_train2_tf)

In [120]:
best_model=clf

In [123]:
report_LR = classification_report(y_train2, Y_train2_pred)
print("Test Report para RegLog \n", report_LR)
print("Confusion Matrix para RegLog\n", confusion_matrix(y_train2, Y_train2_pred))

Test Report para RegLog 
                                                         precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       0.96      0.93      0.94       471
                           Comunicación y Colaboración       0.93      0.90      0.91      1874
                              Correo y Cuenta Uniandes       0.94      0.96      0.95      8451
          Cómputo de alto desempeño para Investigación       0.98      0.96      0.97       524
                                   Entornos de Trabajo       0.93      0.96      0.94      5382
                      Plataformas de enseñanza virtual       0.96      0.97      0.97      4111
Seguimiento y Control del Uso de Licencias de Software       0.88      0.90      0.89       856
                                             Seguridad       0.91      0.84      0.88       525
                    Servicios Académico Administrativo       0.95      0.94      0.95       893
             

- SVM: Sin balanceo de clases 

In [124]:
#SVM no bal
svm = SVC()
svm.fit(X_train2_tf, y_train2)
Y_train2_pred_svm = svm.predict(X_train2_tf)

In [125]:
report_SVM = classification_report(y_train2, Y_train2_pred_svm)
print("Test Report para SVM \n", report_SVM)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train2, Y_train2_pred_svm))

Test Report para SVM 
                                                         precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       0.96      0.90      0.93       471
                           Comunicación y Colaboración       0.94      0.91      0.93      1874
                              Correo y Cuenta Uniandes       0.93      0.97      0.95      8451
          Cómputo de alto desempeño para Investigación       0.98      0.96      0.97       524
                                   Entornos de Trabajo       0.92      0.97      0.94      5382
                      Plataformas de enseñanza virtual       0.96      0.97      0.96      4111
Seguimiento y Control del Uso de Licencias de Software       0.85      0.88      0.86       856
                                             Seguridad       0.94      0.81      0.87       525
                    Servicios Académico Administrativo       0.96      0.94      0.95       893
                

- SVM: Con balanceo de clases 

In [126]:
#SVM bal
y_train2_bal = y_train2_bal.str.replace('Entornos de trabajo \(tiempos sujetos a variación por protocolos de bioseguridad\)','Entornos de Trabajo')
svm = SVC()
svm.fit(X_train2_tf_bal, y_train2_bal)
Y_train2_pred_svm_bal = svm.predict(X_train2_tf_bal)

In [127]:
report_SVM_bal = classification_report(y_train2_bal, Y_train2_pred_svm_bal)
print("Test Report para SVM \n", report_SVM_bal)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train2_bal, Y_train2_pred_svm_bal))

Test Report para SVM 
                                                               precision    recall  f1-score   support

                      Acceso a Internet y Redes Uniandes       0.99      1.00      0.99      1866
                             Comunicación y Colaboración       0.95      0.97      0.96      1866
                                Correo y Cuenta Uniandes       0.97      0.95      0.96      1866
            Cómputo de alto desempeño para Investigación       1.00      1.00      1.00      1866
                                     Entornos de trabajo       0.98      0.94      0.96      1866
                        Plataformas de enseñanza virtual       0.99      0.98      0.98      1866
  Seguimiento y Control del Uso de Licencias de Software       0.97      1.00      0.98      1866
                                               Seguridad       0.99      0.99      0.99      1866
                      Servicios Académico Administrativo       0.99      0.99      0.99   

- SVM: Con reduccion

In [140]:
#SVM reducido
svm = SVC()
svm.fit(X_train2_tf_tsvd, y_train2)
Y_train2_pred_svm_tsvd = svm.predict(X_train2_tf_tsvd)

In [141]:
report_SVM_tsvd = classification_report(y_train2, Y_train2_pred_svm_tsvd)
print("Test Report para SVM \n", report_SVM_tsvd)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train2, Y_train2_pred_svm_tsvd))

Test Report para SVM 
                                                         precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       0.96      0.87      0.92       471
                           Comunicación y Colaboración       0.94      0.87      0.91      1874
                              Correo y Cuenta Uniandes       0.92      0.97      0.94      8451
          Cómputo de alto desempeño para Investigación       0.98      0.94      0.96       524
                                   Entornos de Trabajo       0.90      0.97      0.93      5382
                      Plataformas de enseñanza virtual       0.96      0.97      0.96      4111
Seguimiento y Control del Uso de Licencias de Software       0.87      0.84      0.85       856
                                             Seguridad       0.94      0.77      0.84       525
                    Servicios Académico Administrativo       0.96      0.90      0.93       893
                

- SVM: Con balanceo de clases con reducción

In [6]:
#SVM bal reducido
svm = SVC()
svm.fit(X_train2_tf_bal, y_train2_bal)
Y_train2_pred_svm_tf_bal = svm.predict(X_train2_tf_bal)

In [8]:
report_SVM_tf_bal= classification_report(y_train2_bal, Y_train2_pred_svm_tf_bal)
print("Test Report para SVM \n", report_SVM_tf_bal)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train2_bal, Y_train2_pred_svm_tf_bal))

Test Report para SVM 
                                                           precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       0.99      1.00      0.99      1866
                           Comunicación y Colaboración       0.96      0.98      0.97      1866
                              Correo y Cuenta Uniandes       0.97      0.95      0.96      1866
          Cómputo de alto desempeño para Investigación       1.00      1.00      1.00      1866
                                   Entornos de trabajo       0.97      0.94      0.96      1866
                      Plataformas de enseñanza virtual       0.99      0.98      0.98      1866
Seguimiento y Control del Uso de Licencias de Software       0.97      0.99      0.98      1866
                                             Seguridad       0.98      0.99      0.99      1866
                    Servicios Académico Administrativo       0.99      0.99      0.99      1866
              

- Clasificador bayesiano: sin balanceo de clases ni reduccion de dimensionalidad

In [9]:
#Bayes
nb = MultinomialNB()
nb.fit(X_train2_tf, y_train2)
Y_train2_pred_nb = nb.predict(X_train2_tf)

In [10]:
report_nb = classification_report(y_train2, Y_train2_pred_nb)
print("Test Report para NB \n", report_nb)
print("Confusion Matrix para NB\n", confusion_matrix(y_train2, Y_train2_pred_nb))

Test Report para NB 
                                                            precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       0.95      0.17      0.29       471
                           Comunicación y Colaboración       0.93      0.26      0.40      1874
                              Correo y Cuenta Uniandes       0.71      0.90      0.79      8451
          Cómputo de alto desempeño para Investigación       0.98      0.41      0.57       524
                                   Entornos de trabajo       0.61      0.95      0.74      5382
                      Plataformas de enseñanza virtual       0.78      0.91      0.84      4111
Seguimiento y Control del Uso de Licencias de Software       0.88      0.13      0.23       856
                                             Seguridad       0.92      0.13      0.22       525
                    Servicios Académico Administrativo       0.86      0.38      0.52       893
              

- Clasificador bayesiano: con balanceo de clases sin reduccion de dimensionalidad

In [11]:
#Bayes bal
nb = MultinomialNB()
nb.fit(X_train2_tf_bal, y_train2_bal)
Y_train2_pred_nb_bal = nb.predict(X_train2_tf_bal)

In [12]:
report_nb_bal = classification_report(y_train2_bal, Y_train2_pred_nb_bal)
print("Test Report para NB \n", report_nb_bal)
print("Confusion Matrix para NB\n", confusion_matrix(y_train2_bal, Y_train2_pred_nb_bal))

Test Report para NB 
                                                           precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       0.96      0.92      0.94      1866
                           Comunicación y Colaboración       0.83      0.82      0.83      1866
                              Correo y Cuenta Uniandes       0.88      0.74      0.80      1866
          Cómputo de alto desempeño para Investigación       0.97      0.98      0.98      1866
                                   Entornos de trabajo       0.89      0.77      0.83      1866
                      Plataformas de enseñanza virtual       0.85      0.90      0.88      1866
Seguimiento y Control del Uso de Licencias de Software       0.87      0.92      0.89      1866
                                             Seguridad       0.87      0.90      0.89      1866
                    Servicios Académico Administrativo       0.90      0.86      0.88      1866
               

### 4.2 Optimización de hiperparámetros

Se realiza un procedimiento de optimización de hiperparámetros para regresión logística

In [132]:
pipe = Pipeline([('clf' , RandomForestClassifier())])

param_grid = [
    {'clf' : [LogisticRegression()],
     'clf__penalty' : ['l1', 'l2'],
    'clf__C' : np.logspace(-4, 4, 20),
    'clf__solver' : ['liblinear']},
    {'clf' : [RandomForestClassifier()],
    'clf__n_estimators' : list(range(10,101,10)),
    'clf__max_features' : list(range(6,32,5))}
]
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
clf = GridSearchCV(pipe, param_grid = param_grid, cv = cv, verbose=True, n_jobs=-1, scoring='accuracy')


In [133]:
clf.fit(X_train2_tf, y_train2)
best_logistic_model = clf.best_estimator_
Y_train2_best_logistic_model_pred = best_logistic_model.predict(X_train2_tf)

Fitting 15 folds for each of 100 candidates, totalling 1500 fits


In [134]:
clf.best_params_

{'clf': LogisticRegression(C=1.623776739188721, penalty='l1', solver='liblinear'),
 'clf__C': 1.623776739188721,
 'clf__penalty': 'l1',
 'clf__solver': 'liblinear'}

In [135]:
report_LR_cv = classification_report(y_train2, Y_train2_best_logistic_model_pred)
print("Test Report para RegLog \n", report_LR_cv)
print("Confusion Matrix para RegLog\n", confusion_matrix(y_train2, Y_train2_best_logistic_model_pred))

Test Report para RegLog 
                                                         precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       0.95      0.88      0.91       471
                           Comunicación y Colaboración       0.89      0.86      0.88      1874
                              Correo y Cuenta Uniandes       0.92      0.95      0.93      8451
          Cómputo de alto desempeño para Investigación       0.96      0.96      0.96       524
                                   Entornos de Trabajo       0.91      0.94      0.93      5382
                      Plataformas de enseñanza virtual       0.95      0.96      0.96      4111
Seguimiento y Control del Uso de Licencias de Software       0.83      0.86      0.85       856
                                             Seguridad       0.87      0.79      0.83       525
                    Servicios Académico Administrativo       0.94      0.92      0.93       893
             

Se realiza un procedimiento de optimización de hiperparámetros para SVM

In [136]:
pipe_SVM = Pipeline([('svm' , SVC())])
param_grid_SVM = dict(svm__C = [0.1, 10, 100],
               svm__gamma = ['scale'],
               svm__kernel = ['rbf', 'sigmoid'],
               )

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search_SVM = GridSearchCV(pipe_SVM, param_grid_SVM, cv = cv, verbose=True, n_jobs = -1,scoring='accuracy')

In [137]:
grid_search_SVM.fit(X_train2_tf_bal, y_train2_bal)
best_model_SVM = grid_search_SVM.best_estimator_
Y_pred_SVM_best_model = best_model_SVM.predict(X_train2_tf_bal)

Fitting 15 folds for each of 6 candidates, totalling 90 fits


In [138]:
grid_search_SVM.best_params_

{'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}

In [139]:
report_SVM_bal_best_model = classification_report(y_train2_bal, Y_pred_SVM_best_model)
print("Test Report para SVM \n", report_SVM_bal_best_model)
print("Confusion Matrix para SVM\n", confusion_matrix(y_train2_bal, Y_pred_SVM_best_model))

Test Report para SVM 
                                                            precision    recall  f1-score   support

                    Acceso a Internet y Redes Uniandes       1.00      1.00      1.00      1866
                           Comunicación y Colaboración       1.00      1.00      1.00      1866
                              Correo y Cuenta Uniandes       1.00      0.99      1.00      1866
          Cómputo de alto desempeño para Investigación       1.00      1.00      1.00      1866
                                   Entornos de trabajo       1.00      1.00      1.00      1866
                      Plataformas de enseñanza virtual       1.00      1.00      1.00      1866
Seguimiento y Control del Uso de Licencias de Software       1.00      1.00      1.00      1866
                                             Seguridad       1.00      1.00      1.00      1866
                    Servicios Académico Administrativo       1.00      1.00      1.00      1866
             

### 4.3 Entrenamiento Red Neuronal FastText

In [307]:
model2_ft = fasttext.train_supervised(input='train2_ft.txt', autotuneValidationFile='val2_ft.txt')

In [382]:
model2_ft.test("val2_ft.txt")

(6999, 0.7755393627661095, 0.7755393627661095)

In [11]:
y_train2_pred = X_train2_ft['preprocessed_ad'].apply(lambda row: model2_ft.predict(row)[0][0])

In [387]:
y_train2_ft = y_train2_ft.str.replace('\_\(tiempos_sujetos_a_variación_por_protocolos_de_bioseguridad\)','', case=False)
y_train2_pred = y_train2_pred.str.replace('\_\(tiempos_sujetos_a_variación_por_protocolos_de_bioseguridad\)','', case=False)

In [401]:
#Información y estadísticos
report_test = classification_report(y_train2_ft, y_train2_pred)

print("Test Report para Mejor Modelo\n", report_test)
print("Confusion Matrix para Mejor Modelo\n", confusion_matrix(y_train2_ft, y_train2_pred))

Test Report para Mejor Modelo
                                                                  precision    recall  f1-score   support

                    __label__Acceso_a_Internet_y_Redes_Uniandes       0.68      0.74      0.71       353
                           __label__Comunicación_y_Colaboración       0.68      0.81      0.74      1405
                              __label__Correo_y_Cuenta_Uniandes       0.85      0.88      0.86      6338
          __label__Cómputo_de_alto_desempeño_para_Investigación       0.88      0.80      0.84       393
                                   __label__Entornos_de_trabajo       0.82      0.86      0.84      4036
                      __label__Plataformas_de_enseñanza_virtual       0.92      0.88      0.90      3083
__label__Seguimiento_y_Control_del_Uso_de_Licencias_de_Software       0.71      0.70      0.70       642
                                             __label__Seguridad       0.69      0.59      0.63       394
                    __l

## Validación

#### Modelo A

El mejor modelo para Tipo es el SVM con optimización de hiperparámetros

In [104]:
#Preprocesamiento de los datos
X_test1 = preprocessing(X_test1)
X_test1['tokenized_ad'] = X_test1.apply(lambda row: nltk.word_tokenize(row['preprocessed_ad']), axis=1)
X_test1['preprocessed_ad'] = X_test1.apply(lambda row: " ".join(preprocessing2(row['tokenized_ad'])),axis=1)
X_test1['lem_ad'] = X_test1['preprocessed_ad'].apply(lambda row: " ".join([w.lemma_ for w in nlp(str(row))]))
X_test1['tokenized_ad'] = X_test1.apply(lambda row: nltk.word_tokenize(row['lem_ad']), axis=1)
X_test1['stem_ad'] = X_test1.apply(lambda row: stem_words(row['tokenized_ad']),axis=1)

In [107]:
X_test_text1_m = X_test1[['stem_ad']]
X_test_text1 = X_test_text1_m['stem_ad']
X_test_text1 = X_test_text1.apply(lambda x: ''.join(x))
X_test_tf1 = tf_idf1.transform(X_test_text1)
X_test_tf1 = scaler.transform(X_test_tf1)

In [108]:
#Ejecución del modelo
best_model = best_model_SVM
y_test_pred = best_model.predict(X_test_tf1)

In [109]:
#Información y estadísticos
report_test = classification_report(y_test1, y_test_pred)

print("Test Report para Mejor Modelo\n", report_test)
print("Confusion Matrix para Mejor Modelo\n", confusion_matrix(y_test1, y_test_pred))


Test Report para Mejor Modelo
               precision    recall  f1-score   support

   Incidente       0.00      0.00      0.00      2845
   Solicitud       0.59      1.00      0.74      4154

    accuracy                           0.59      6999
   macro avg       0.30      0.50      0.37      6999
weighted avg       0.35      0.59      0.44      6999

Confusion Matrix para Mejor Modelo
 [[   0 2845]
 [   0 4154]]


#### Modelo B

El mejor modelo obtenido es la red neuronal de fasttext

In [314]:
#Preprocesamiento de los datos
X_test2_ft = preprocessing(X_test2_ft)
test2_ft = y_test2_ft + ' ' + X_test2_ft['preprocessed_ad']
test2_ft.to_csv('test2_ft.txt',header=None, index=None, sep=' ', escapechar=" ", quoting=csv2.QUOTE_NONE, encoding='utf-8')

In [315]:
model2_ft.test("test2_ft.txt")

(6999, 0.7739677096728104, 0.7739677096728104)

In [391]:
y_test2_pred = X_test2_ft['preprocessed_ad'].apply(lambda row: model2_ft.predict(row)[0][0])

In [360]:
y_test2_pred

14931            __label__Plataformas_de_enseñanza_virtual
12443            __label__Plataformas_de_enseñanza_virtual
21006    __label__Entornos_de_trabajo_(tiempos_sujetos_...
24704    __label__Entornos_de_trabajo_(tiempos_sujetos_...
30861                   __label__Servicios_Administrativos
                               ...                        
34190    __label__Entornos_de_trabajo_(tiempos_sujetos_...
14951            __label__Plataformas_de_enseñanza_virtual
39004                    __label__Correo_y_Cuenta_Uniandes
26484                    __label__Correo_y_Cuenta_Uniandes
18946                    __label__Correo_y_Cuenta_Uniandes
Name: preprocessed_ad, Length: 6999, dtype: object

In [392]:
y_test2_ft = y_test2_ft.str.replace('\_\(tiempos_sujetos_a_variación_por_protocolos_de_bioseguridad\)','', case=False)
y_test2_pred = y_test2_pred.str.replace('\_\(tiempos_sujetos_a_variación_por_protocolos_de_bioseguridad\)','', case=False)

In [393]:
#Información y estadísticos
report_test = classification_report(y_test2_ft, y_test2_pred)

print("Test Report para Mejor Modelo\n", report_test)
print("Confusion Matrix para Mejor Modelo\n", confusion_matrix(y_test2_ft, y_test2_pred))

Test Report para Mejor Modelo
                                                                  precision    recall  f1-score   support

                    __label__Acceso_a_Internet_y_Redes_Uniandes       0.71      0.79      0.75       118
                           __label__Comunicación_y_Colaboración       0.67      0.80      0.73       468
                              __label__Correo_y_Cuenta_Uniandes       0.84      0.87      0.85      2113
          __label__Cómputo_de_alto_desempeño_para_Investigación       0.89      0.77      0.83       131
                                   __label__Entornos_de_trabajo       0.81      0.87      0.84      1345
                      __label__Plataformas_de_enseñanza_virtual       0.92      0.88      0.90      1028
__label__Seguimiento_y_Control_del_Uso_de_Licencias_de_Software       0.68      0.66      0.67       214
                                             __label__Seguridad       0.71      0.58      0.64       131
                    __l